In [1]:
tnt = require 'torchnet'
image = require 'image'

In [13]:
train = torch.load('../data/train.t7')
test = torch.load('../data/test.t7')
DATA_PATH = "/home/anirudhan/workspace/traffic-sign-detection/data"
WIDTH, HEIGHT = 32, 32

In [14]:
function resize(img)
    return image.scale(img, WIDTH,HEIGHT)
end

function transformInput(inp)
    f = tnt.transform.compose{
        [1] = resize
    }
    return f(inp)
end

In [15]:
function getTrainSample(dataset, idx)
    r = dataset[idx]
    classId, track, file = r[9], r[1], r[2]
    file = string.format("%05d/%05d_%05d.ppm", classId, track, file)
    return transformInput(image.load(DATA_PATH .. '/train_images/'..file))
end

function getTrainLabel(dataset, idx)
    return dataset[idx][9]
end

In [16]:
function getTestSample(dataset, idx)
    r = dataset[idx]
    file = DATA_PATH .. "/test_images/" .. string.format("%05d.ppm", r[1])
    return transformInput(image.load(file))
end

In [17]:
trainDataset = tnt.SplitDataset{
    partitions = {train=0.9, val=0.1},
    dataset = tnt.ShuffleDataset{
        dataset = tnt.ListDataset{
            list = torch.range(1, train:size(1)):long(),
            load = function(idx)
                return {
                    input =  getTrainSample(train, idx),
                    target = getTrainLabel(train, idx)
                }
            end
        }
    }
}

In [18]:
function getIterator(dataset)
    return tnt.ParallelDatasetIterator{
        nthread = 1,
        init = function() require 'torchnet' end,
        closure = function()
            return tnt.BatchDataset{
                batchsize = 128,
                dataset = dataset
            }
        end
    }
end

In [29]:
trainDataset:select('val')

In [30]:
trainDataset:size()

3920	


In [31]:
35288 + 3920

39208	


In [32]:
train:size()

 39209
     9
[torch.LongStorage of size 2]

